In [1]:
import altair as alt
import pandas as pd
import numpy as np
import pickle
import scipy.stats

from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier

from scipy.io import loadmat
import glob

In [2]:
def load_learning_curves(path):
    lcs = []
    for f in glob.glob(f"{path}/learning_curve_*continued.mat"):
        d = loadmat(f)
        lcs.append(d["totalReward"][0])
    lcs = np.array(lcs)
    mean_lc = lcs.mean(axis=0)
    lc_std = lcs.std(axis=0)
    return mean_lc, lc_std

In [3]:
d = []
d.extend(load_learning_curves("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/"))
d.extend(load_learning_curves("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/"))
d.extend(load_learning_curves("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/"))
d.extend(load_learning_curves("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/"))
d.extend(load_learning_curves("models_20/log_decay/N_6_vc_1_vm_1_lr_0.001/"))
d = pd.DataFrame(d).T
d.columns = ["N_6_vc_3_vm_1", "N_6_vc_3_vm_1_std",
             "N_6_vc_3_vm_3", "N_6_vc_3_vm_3_std",
             "N_6_vc_3_vm_1_f2", "N_6_vc_3_vm_1_f2_std",
             "N_6_vc_3_vm_3_f2", "N_6_vc_3_vm_3_f2_std",
             "N_6_vc_1_vm_1", "N_6_vc_1_vm_1_std",
            ]
d = d.reset_index()
d = d.iloc[:6000]
d

,index,N_6_vc_3_vm_1,N_6_vc_3_vm_1_std,N_6_vc_3_vm_3,N_6_vc_3_vm_3_std,N_6_vc_3_vm_1_f2,N_6_vc_3_vm_1_f2_std,N_6_vc_3_vm_3_f2,N_6_vc_3_vm_3_f2_std,N_6_vc_1_vm_1,N_6_vc_1_vm_1_std
0,0,-2.158376e+09,2.777453e+08,-2.104567e+09,2.649213e+08,-2.174377e+09,3.475372e+08,-2.022540e+09,3.075923e+08,-4.590123e+09,0.0
1,1,-2.152792e+09,2.774958e+08,-2.102347e+09,2.593964e+08,-2.172509e+09,3.462488e+08,-2.019543e+09,2.957430e+08,-4.578164e+09,0.0
2,2,-2.142409e+09,2.763840e+08,-2.095255e+09,2.544240e+08,-2.158091e+09,3.340554e+08,-2.016380e+09,2.846860e+08,-4.568551e+09,0.0
3,3,-2.123607e+09,2.732609e+08,-2.081905e+09,2.458953e+08,-2.140166e+09,3.230842e+08,-2.010833e+09,2.772901e+08,-4.499745e+09,0.0
4,4,-2.108881e+09,2.665928e+08,-2.066644e+09,2.463304e+08,-2.117490e+09,3.122396e+08,-2.002048e+09,2.732130e+08,-4.415919e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5995,5995,-7.818147e+06,6.757311e+06,-6.324448e+06,8.620437e+06,-7.935473e+06,7.604362e+06,-4.018934e+06,5.677968e+06,-6.442305e+07,0.0
5996,5996,-7.675579e+06,6.560626e+06,-6.152289e+06,8.176879e+06,-7.714011e+06,7.278016e+06,-3.913422e+06,5.535258e+06,-6.229302e+07,0.0
5997,5997,-7.690122e+06,6.363037e+06,-5.925697e+06,7.764787e+06,-7.767473e+06,7.358321e+06,-4.001823e+06,5.822030e+06,-6.000338e+07,0.0
5998,5998,-7.558415e+06,6.231544e+06,-5.888982e+06,7.332643e+06,-7.521436e+06,7.021057e+06,-3.886056e+06,5.635758e+06,-5.800668e+07,0.0


In [4]:
d["N_6_vc_3_vm_3"].mean(), d["N_6_vc_3_vm_1"].mean()

(-29680416.11254453, -28315295.76676146)

In [5]:
d["N_6_vc_3_vm_3_f2"].mean(), d["N_6_vc_3_vm_1_f2"].mean() 

(-34543040.14208828, -38117372.782840624)

In [6]:
d1 = d.rolling(100).mean().iloc[100::100,:]
d1 = d1.reset_index()
line1 = alt.Chart(d1).mark_line(color="orange").encode(
    x='index',
    y=alt.Y('N_6_vc_3_vm_1', scale=alt.Scale(type="log"))
)

line2 = alt.Chart(d1).mark_line().encode(
    x='index',
    y=alt.Y('N_6_vc_3_vm_3', scale=alt.Scale(type="log"))
)

line1 + line2

alt.LayerChart(...)

In [7]:
d1 = d.rolling(100).mean().iloc[100::100,:]
d1 = d1.reset_index()
line1a = alt.Chart(d1).mark_line(color="orange").encode(
    x='index',
    y=alt.Y('N_6_vc_3_vm_1_f2', scale=alt.Scale(type="log"))
)

line2a = alt.Chart(d1).mark_line().encode(
    x='index',
    y=alt.Y('N_6_vc_3_vm_3_f2', scale=alt.Scale(type="log"))
)

line3a = alt.Chart(d1).mark_line().encode(
    x='index',
    y=alt.Y('N_6_vc_1_vm_1', scale=alt.Scale(type="log"))
)

line1 + line2 + line1a + line2a + line3a

alt.LayerChart(...)

In [8]:
def symmetric_uncertainty(X, Y):
    """
    Symmetric uncertainty is mutual information normalized to 0-1 interval.
    """
    _, counts = np.unique(X, return_counts=True)
    probs_X = counts / counts.sum()
    
    _, counts = np.unique(Y, return_counts=True)
    probs_Y = counts / counts.sum()

    XY = np.vstack([X, Y]).T
    _, counts = np.unique(XY, axis=0, return_counts=True)
    probs = counts / counts.sum()

    H_X = scipy.stats.entropy(probs_X, base=2)
    H_Y = scipy.stats.entropy(probs_Y, base=2)
    H_XY = scipy.stats.entropy(probs, base=2)

    I_XY = H_X + H_Y - H_XY

    return 2 * I_XY / (H_X + H_Y)

In [9]:
def conditional_entropy(X, Y):
    """
    Returns counterpart of normalized conditional entropy: 1 - H(X|Y) / H(X)
    """    
    _, counts = np.unique(X, return_counts=True)
    probs_X = counts / counts.sum()
        
    _, counts = np.unique(Y, return_counts=True)
    probs_Y = counts / counts.sum()

    XY = np.vstack([X, Y]).T
    idx, counts = np.unique(XY, axis=0, return_counts=True)
    probs = counts / counts.sum()
    
    H_XY = -np.sum(probs*(np.log2(probs)))
    H_Y = -np.sum(probs_Y*(np.log2(probs_Y)))
    H_X = -np.sum(probs_X*(np.log2(probs_X)))

    return 1 - (H_XY - H_Y) / H_X

In [10]:
list(range(10))[0:-1:2][1:], list(range(10))[0:-2:2]

([2, 4, 6, 8], [0, 2, 4, 6])

In [11]:
def calc_uncertaintes(path, freq=1):
    with open(path, "rb") as f:
        data = pickle.load(f)
    states = data["childStates"]
    cs = data["childSignals"]
    ms = data["motherSignals"]
    ma = data["motherActions"]
    
    states_ = states[:,0::freq]
    cs_ = cs[:,0::freq]
    ms_ = ms[:,0::freq]
    ma_ = ma[:,0::freq]
    
    ms_1 = ms[:,:-1][:,0::freq]
    if freq > 1:
        ms_1 = ms_1[:,:-1]
       
    d = defaultdict(list)
    for i in range(cs.shape[0]):       
        d["action | child_s"].append(conditional_entropy(ma_[i, 1:].flatten(), cs_[i, 1:].flatten()))                            
        d["action | care_s"].append(conditional_entropy(ma[i].flatten(), ms[i].flatten()))
        d["action | action"].append(conditional_entropy(ma[i, 1:].flatten(), ma[i, :-1].flatten()))      
        d["care_s | child_s"].append(conditional_entropy(ms_[i].flatten(), cs_[i].flatten()))
        d["care_s | care_s"].append(conditional_entropy(ms[i, 1:].flatten(), ms[i, :-1].flatten()))
        d["child_s | child_s"].append(conditional_entropy(cs_[i, 1:].flatten(), cs_[i, :-1].flatten()))
        d["child_s | care_s"].append(conditional_entropy(cs_[i, 1:].flatten(), ms_1[i].flatten()))
    return pd.DataFrame(d)

In [14]:
d1 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/test_trajectories.pkl")
d2 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/test_trajectories.pkl")
d3 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", freq=2)
d4 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", freq=2)
#d5 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_3_mActFreq_1/test_trajectories.pkl", freq=3)
#d6 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_3_mActFreq_1/test_trajectories.pkl", freq=3)
#d7 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_4_mActFreq_1/test_trajectories.pkl", freq=4)
#d8 = calc_uncertaintes("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_4_mActFreq_1/test_trajectories.pkl", freq=4)

d1["condition"] = "f = 1, dialogue"
d2["condition"] = "f = 1, no dialogue"
d3["condition"] = "f = 1/2, dialogue"
d4["condition"] = "f = 1/2, no dialogue"
#d5["condition"] = "f3_dialog"
#d6["condition"] = "f3_no_dialog"
#d7["condition"] = "f4_dialog"
#d8["condition"] = "f4_no_dialog"

d = pd.concat([d1, d2, d3, d4], axis=0)
d = d.reset_index().iloc[:, 1:]

d1 = d.melt(value_vars=d.columns[:-1], id_vars="condition")
d1.columns = ["condition", "variable pair", "relation strength"]
alt.Chart(d1).mark_boxplot(extent='min-max').encode(x='variable pair', y='relation strength', column='condition').configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_header(
    labelFontSize=14,
    titleFontSize=16
)

/tmp/ipykernel_13430/2769261706.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (H_XY - H_Y) / H_X


alt.Chart(...)

In [40]:
def calc_positional_uncertaintes(path, freq=1):
    with open(path, "rb") as f:
        data = pickle.load(f)
    states = data["childStates"]
    cs = data["childSignals"]
    ms = data["motherSignals"]
    ma = data["motherActions"]
    
    states_ = states[:,0::freq]
    cs_ = cs[:,0::freq]
    ms_ = ms[:,0::freq]
    ma_ = ma[:,0::freq]
          
    d = []
    print(ma[0].shape[0])
    for i in range(cs.shape[0]):
        #for j in range(ma_[i].shape[0]):
        #    d.append(("ma_vs_cs", j, conditional_entropy(ma_[i,j], cs_[i,j])))
        #    d.append(("ms_vs_cs", j, conditional_entropy(ms_[i,j], cs_[i,j])))
        #for j in range(ma[i].shape[0]-1):
        #    d.append(("ma_vs_ms", j, conditional_entropy(ma[i,j], ms[i,j])))
        for j in range(ma[i].shape[0]-1):
            d.append(("action | action", j+1, conditional_entropy(ma[i,j+1], ma[i,j])))
        #for j in range(ma[i].shape[0]-2):
        #    d.append(("ma_vs_ma+2", j, conditional_entropy(ma[i,j+2], ma[i,j])))
        #for j in range(cs_[i].shape[0]-1):
        #    d.append(("ma+1_vs_cs", j, conditional_entropy(ma[i,j*freq+1], cs_[i,j])))
        #for j in range(cs_[i].shape[0]-2):
        #    d.append(("ma+2_vs_cs", j, conditional_entropy(ma[i,j*freq+2], cs_[i,j])))
        #for j in range(cs_[i].shape[0]-1):
        #    d.append(("ms+1_vs_cs", j, conditional_entropy(ms[i,j*freq+1], cs_[i,j])))
        #for j in range(cs_[i].shape[0]-1):
        #    d.append(("cs+1_vs_ms", j, conditional_entropy(cs_[i,j+1], ms[i,(j+1)*freq-1])))
    d = pd.DataFrame(d)
    d.columns = ["variable pair", "round", "relation strength"]
    return d

In [41]:
d = calc_positional_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/test_trajectories.pkl", freq=1)
alt.Chart(d).mark_boxplot(extent='min-max').encode(x='round', y='relation strength').facet(facet='variable pair', columns=3)

20


alt.FacetChart(...)

In [43]:
d = calc_positional_uncertaintes("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", freq=2)

alt.Chart(d).mark_boxplot(extent='min-max').encode(x='round', y='relation strength').facet(facet='variable pair', columns=3).configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_header(
    labelFontSize=14,
    titleFontSize=16
)

20


alt.FacetChart(...)

In [36]:
d = calc_positional_uncertaintes("models_20/log_decay/N_6_vc_3_vm_1_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", freq=2)
alt.Chart(d).mark_boxplot(extent='min-max').encode(x='turn', y='value').facet(facet='relation', columns=3)

/tmp/ipykernel_2567/4173649470.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (H_XY - H_Y) / H_X


alt.FacetChart(...)

In [20]:
def calc_rf_importances(path):

    with open(path, "rb") as f:
        data = pickle.load(f)
    states = data["childStates"]
    cs = data["childSignals"]
    ms = data["motherSignals"]
    ma = data["motherActions"]

    print(cs[0, 0, :20])
    
    clf = RandomForestClassifier(n_estimators=100)

    importances = []
    scores = []

    for i in range(cs.shape[0]):
        #y = ma[i, 3:].flatten()
        y = ms[i, 3:].flatten()
        X = np.vstack([#ms[i, 3:].flatten(),
                       cs[i, 3:].flatten(),
                       #ms[i, 2:-1].flatten(),
                       cs[i, 2:-1].flatten(),
                       #ms[i, 1:-2].flatten(),
                       cs[i, 1:-2].flatten(),
                       #ms[i, :-3].flatten(),
                       cs[i, :-3].flatten(),
                       ma[i, 3:].flatten(),
                       ma[i, 1:-2].flatten(),
                       ma[i, :-3].flatten(),
                    ]).T
        clf.fit(X, y)
        importances.append(clf.feature_importances_)
        scores.append(clf.score(X, y))
    
    print(np.mean(scores))
    
    d = pd.DataFrame(importances)
    d.columns = [#"Ms",
                 "Cs",
                 #"Ms-1",
                 "Cs-1",
                 #"Ms-2",
                 "Cs-2",
                 #"Ms-3",
                 "Cs-3",
                 "Ma-3", "Ma-2", "Ma-1"
    ]
    
    d1 = d.melt(value_vars=d.columns)
    return alt.Chart(d1).mark_boxplot(extent='min-max').encode(x='variable', y='value')

In [27]:
calc_rf_importances("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_1_mActFreq_1/test_trajectories.pkl")

[2 2 1 2 0 0 0 0 2 0 2 0 2 1 2 2 1 1 1 2]
0.557


/home/zubekj/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [26]:
calc_rf_importances("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl")

[2 2 1 0 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2]
0.546029411764706


/home/zubekj/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [144]:
cs.shape

(10, 20, 500)

In [51]:
def cs_to_ma(path, model_idx, lag=2):
    with open(path, "rb") as f:
        data = pickle.load(f)
    states = data["childStates"]
    cs = data["childSignals"]
    ms = data["motherSignals"]
    ma = data["motherActions"]

    n = cs.shape[1]
    
    cs_cs_1 = np.vstack([cs[model_idx, i:(n-lag+1+i)].flatten() for i in range(lag)]).T
  
    dist_action = []

    for i in np.unique(ma[model_idx, (lag-1):]):
        d = pd.DataFrame(cs_cs_1[ma[model_idx, (lag-1):].flatten() == i])
        d.columns = [f"Cs{-j}" for j in range(lag)]
        dist_action.append(d.groupby(list(d.columns)).size())

    d = pd.DataFrame(dist_action).T
    
    p = d.sum(axis=0)
    p /= p.sum()
    H_Y = -(np.log(p)*p).sum()
    
    p = d.sum(axis=1)
    p /= p.sum()
    H_X = -(np.log(p)*p).sum()
    
    p = d / d.sum().sum()
    H_XY = -(np.log(p)*p).sum().sum()
    
    mi = H_X + H_Y - H_XY
    print(mi)
    
    return d

In [57]:
cs_to_ma("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", 1, 1)

0.5307674814120986


,0,1,2,3,4,5,6
Cs0,,,,,,,
0,942,5014,57,348,5395,330,218
1,868,514,6859,6452,348,362,627
2,2299,1494,395,963,441,5759,315


In [58]:
cs_to_ma("models_20/log_decay/N_6_vc_3_vm_3_lr_0.001_hc_50_hm_50_cSpeakFreq_2_mActFreq_1/test_trajectories.pkl", 1, 2)

0.7533421091671633


0     1     2     3     4     5    6
Cs0 Cs-1                                         
0   0      789  4855    19   333  1490   216   83
    1      184   106    33  1862     1     8   84
    2       48    40    51    13    35  1435   19
1   0       68    74    13     5  1973    62   57
    1      454   238  6783  2279   337   336  436
    2      116   161    32    25    66  1621   41
2   0       50    45    13     2  1386    34   36
    1      204   126    27  1964     5    10   65
    2     2093  1255   251   892   287  2143  231